In [1]:
db = 'gthruth_valeflores_v3'

table = '/mnt/d/autocls_e3/classification_exp31.xlsx'
sheet = 'experiments_list'

ref_folder = '/home/jasp/autocls/classref/'
res_folder = '/home/jasp/autocls/results/'
val_table = '/mnt/d/autocls_e3/valida/experiments_roads_ref.xlsx'

In [2]:
import os
import pandas as pd
from glass.sql.q import q_to_obj
from glass.rst.confmtx.totbl import confmtx_twrst, confmtx_fmrst
from glass.pys.oss import fprop
from glass.rd import tbl_to_obj
from glass.wt import obj_to_tbl

In [3]:
exp = tbl_to_obj(table, sheet=sheet)

exp = exp[exp.status == 'run']

exp.reset_index(inplace=True)

In [4]:
mtxs, sintese = [], []
mtxscols = []
all_classes = []
maincols = ['exp', 'total_area', 'overall_accuracy', 'kappa', 'macro-f1', 'weighted-f1']
for i, row in exp.iterrows():
    classes_df = q_to_obj(db, (
        "SELECT mj.codeval, mj.nameval "
        "FROM expclasses AS mj "
        "INNER JOIN experiments AS jt "
        "ON mj.expfk = jt.eid "
        f"WHERE jt.slug = '{row.expdbname}'"
    ))

    classes_leg = {}

    for _i, r in classes_df.iterrows():
        classes_leg[r.codeval] = r.nameval
    
    mtx = confmtx_fmrst(
        os.path.join(ref_folder, row.ref),
        os.path.join(res_folder, row.classified),
        class_labels=classes_leg
    )

    mtx = mtx.set_index('class')
    
    # Get classes of this matrix
    mtxcols = [c for c in mtx.columns.values if c != 'row_area' and c != 'precision']

    # Get classes considering all matrices
    for _c in mtxcols:
        if _c in all_classes:
            continue

        all_classes.append(_c)

    mtxs.append(mtx)
    mtxscols.append(mtxcols)

# Get sintese table
for i, row in exp.iterrows():
    # Change index - class to index
    cols = mtxscols[i]

    # Get global measures
    over  = mtxs[i].loc['overall_accuracy', cols[0]]
    kappa = mtxs[i].loc['kappa', cols[0]]
    tarea = mtxs[i].loc['total_area', 'row_area']
    mf1   = mtxs[i].loc['macro-f1', cols[0]]
    wf1   = mtxs[i].loc['weighted-f1', cols[0]]

    # Get f1-scores
    f1_scores = mtxs[i].loc['f1-score', cols].to_dict()

    # Get values for all columns
    allcols = []
    for c in all_classes:
        if c in f1_scores:
            allcols.append(f1_scores[c])
        else:
            allcols.append(0)
    
    sintese.append([row['name'], tarea, over, kappa, mf1, wf1] + allcols)

    mtxs[i].insert(0, 'class', mtxs[i].index)

In [5]:
sdf = pd.DataFrame(sintese, columns=maincols + all_classes)

In [6]:
sdf

,exp,total_area,overall_accuracy,kappa,macro-f1,weighted-f1,no_roads,roads
0,exp331,61166.0,91.583559,0.594522,79.607473,90.815541,95.235098,63.979849
1,exp332,61166.0,92.636432,0.646206,82.210337,91.977815,95.829321,68.591353


In [7]:
alldfs = [sdf]
alldfs.extend(mtxs)
sheets = ['sintese'] + exp.name.tolist()

In [8]:
obj_to_tbl(alldfs, val_table, sheetsName=sheets)

'/mnt/d/autocls_e3/valida/experiments_roads_ref.xlsx'